In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas import DataFrame
import sqlite3
import json

In [2]:
url = "https://ca.trustpilot.com/review/www.amazon.com"
req = requests.get(url)
text = req.content

In [3]:
soup = BeautifulSoup(text, 'html5lib')
reviews = soup.find_all(class_='review-card')
reviews

[<div class="review-card">
         
 
 
 
 
 
 <article class="review" data-review-domain-id="46ad346800006400050092d0" data-review-domain-name="www.amazon.com" data-review-is-reported="false" data-review-user-id="60315b094b6bae001a4128b7" data-reviewmid="60315b13f85d7509d8df514b" id="60315b13f85d7509d8df514b">
 
     <script data-initial-state="review-info" type="application/json">
 {"socialShareUrl":"https://ca.trustpilot.com/reviews/60315b13f85d7509d8df514b","businessUnitId":"46ad346800006400050092d0","businessUnitDisplayName":"Amazon","consumerId":"60315b094b6bae001a4128b7","consumerName":"Gary Osborn","reviewId":"60315b13f85d7509d8df514b","reviewHeader":"Amazon is great about taking your money…","reviewBody":"Amazon is great about taking your money through Amazon but when you have a issue with a item it is \u0022Hey that\u0027s a reseller, you have to go through them\u0022. Good luck with that or contacting Amazon customer support you will be buried in links to everything but Ama

In [4]:
review_data = []

for i in range(len(reviews)):
    
    con = reviews[i].script.contents
    
    con1 = json.dumps(con)
    con2 = con1.split(',')
    
    try:
    
        review_data.append(
        {
            'URL': con2[0].strip().split('\\"')[3],
            'business_unit':con2[2].strip().split('\\"')[3],
            'reviewer_id':con2[3].strip().split('\\"')[3],
            'reviewer_name':con2[4].strip().split('\\"')[3],
            'review_header':con2[6].strip().split('\\"')[3],
            'review_body':reviews[i].p.text.strip(),
            'star_rating': [s for s in con2[-1].strip().split('\\"')[2] if s.isdigit()],
            'location':reviews[i].div.contents[3].contents[3].text.strip(),
            'count_review_by_user':reviews[i].div.contents[3].contents[1].text.strip()
        }
        )
    
    except:
        print('invalid review')

In [6]:
r_df = pd.DataFrame(review_data)

In [7]:
r_df

,URL,business_unit,reviewer_id,reviewer_name,review_header,review_body,star_rating,location,count_review_by_user
0,https://ca.trustpilot.com/reviews/60315b13f85d...,Amazon,60315b094b6bae001a4128b7,Gary Osborn,Amazon is great about taking your money\u2026,Amazon is great about taking your money throug...,[1],US,1 review
1,https://ca.trustpilot.com/reviews/60314f28f85d...,Amazon,5357e87f00006400016fad3d,KENNETH CAIRNS,Amazon Let Down By Hermes,No problem with Amazon usually until today. Ha...,[1],GB,11 reviews
2,https://ca.trustpilot.com/reviews/60314d86f85d...,Amazon,58fd17810000ff000a904fd2,Mark Gregory,What on earth is going on with my item\u2026,What on earth is going on with my item because...,[1],GB,11 reviews
3,https://ca.trustpilot.com/reviews/60313c90f85d...,Amazon,60313b74bae32a0019ebd436,gbgb,\ud83d\ude32\u2658 \uff43\uff55\uff53\uff54\u...,"Two items ordered on my account not by me, con...",[2],IN,2 reviews
4,https://ca.trustpilot.com/reviews/60312f9df85d...,Amazon,560938da0000ff0001df072a,Jamie Holder,Two items ordered on my account not by\u2026,"Two items ordered on my account not by me, con...",[1],GB,5 reviews
5,https://ca.trustpilot.com/reviews/60312d98f85d...,Amazon,60312d90d37e8b001a2c1ed4,Harry Leschke,Delivery date never met.,"Amazon puts a delivery date on the order, but ...",[1],US,1 review
6,https://ca.trustpilot.com/reviews/60312bdaf85d...,Amazon,60312bd1d37e8b001a2c1d9a,liam lewis,Prime + 24 hours for an investigation,No matter what I do they always mess up my ord...,[1],GB,1 review
7,https://ca.trustpilot.com/reviews/60311bcdf85d...,Amazon,6031172abb3faf00199565bd,Scot Trekkers,Very happy Prime Member,Absolutely love Amazon. It has been a lifeline...,[5],GB,3 reviews
8,https://ca.trustpilot.com/reviews/6031148af85d...,Amazon,602ba23eb9d941001aeedfac,Mark Richardson,Highly impressed.,Ordered a few items in the January sales and a...,[4],GB,5 reviews
9,https://ca.trustpilot.com/reviews/6030f2eef85d...,Amazon,602fbb834d2c59001b358e0c,J. Kunes,Lost 3 returns,Customer service has hit an all time low.Retur...,[1],US,2 reviews


In [8]:
r_df.to_excel("reviews.xlsx")